In [10]:
from run_calibrations import get_database
from dotenv import load_dotenv
load_dotenv('.env-db')

True

In [11]:
db = get_database()

In [12]:
coll_name = 'queries/contrast/openchat/openchat_v3.1'
db[coll_name].update_many({}, {'$set':{'rating':-1}})